In [1]:
import pandas as pd
import numpy as np

In [4]:
raw = pd.read_csv('서울관광재단_식당품질정보_20230111.csv', encoding = 'cp949')
raw.head()

,식당(ID),식당명,지점명,지역명,어워드정보설명,(RTI)지수,온라인화진행여부,수용태세지수,인기도,트립어드바이저평점,씨트립평점,네이버평점
0,10012,명동정,NaN,중구,모범음식점(2015),4.213839,N,0.370,0.33,NaN,NaN,4.40
1,10014,봉추찜닭,명동중앙우체국점,중구,모범음식점(2019),4.423649,Y,0.330,0.51,NaN,4.7,4.45
2,10015,금계찜닭,명동점,중구,NaN,NaN,Y,0.082,NaN,NaN,NaN,NaN
3,10016,명동한우방,명동본점,중구,NaN,3.922220,N,0.244,0.09,NaN,NaN,4.54
4,10018,삼호복집,NaN,서대문구,모범음식점(2004),4.400069,Y,0.307,0.33,4.0,NaN,4.33


In [5]:
my_area = raw[raw['지역명'].isin(['마포구','금천구','서초구','성동구','동대문구'])][['식당(ID)','식당명', '지점명','지역명','네이버평점']]

my_area.head(10)

,식당(ID),식당명,지점명,지역명,네이버평점
23,10054,옛날집,NaN,마포구,4.37
65,12282,오발탄,서초점,서초구,4.35
66,12283,마장동한우촌,NaN,성동구,4.09
76,12293,태능식당,NaN,마포구,NaN
79,12297,육백더육백,NaN,성동구,4.37
88,12306,매꼬만 명태찜,NaN,금천구,NaN
101,12319,명동칼국수,본점,금천구,4.33
104,12322,인생설렁탕,장한평역점,동대문구,4.33
107,12326,칭닌,NaN,마포구,NaN
121,12341,청량리고기마을,NaN,동대문구,4.34


In [6]:
import numpy as np

# NaN 값을 처리하기 위해 numpy를 import합니다.

my_area['식당명+지점명'] = np.where(
    my_area['지점명'].notna(),
    my_area['식당명'].astype(str) + " " + my_area['지점명'].astype(str),
    my_area['지역명'] + " " + my_area['식당명'].astype(str)
)

my_area.head(10)

,식당(ID),식당명,지점명,지역명,네이버평점,식당명+지점명
23,10054,옛날집,NaN,마포구,4.37,마포구 옛날집
65,12282,오발탄,서초점,서초구,4.35,오발탄 서초점
66,12283,마장동한우촌,NaN,성동구,4.09,성동구 마장동한우촌
76,12293,태능식당,NaN,마포구,NaN,마포구 태능식당
79,12297,육백더육백,NaN,성동구,4.37,성동구 육백더육백
88,12306,매꼬만 명태찜,NaN,금천구,NaN,금천구 매꼬만 명태찜
101,12319,명동칼국수,본점,금천구,4.33,명동칼국수 본점
104,12322,인생설렁탕,장한평역점,동대문구,4.33,인생설렁탕 장한평역점
107,12326,칭닌,NaN,마포구,NaN,마포구 칭닌
121,12341,청량리고기마을,NaN,동대문구,4.34,동대문구 청량리고기마을


In [7]:
name_list = my_area['식당명+지점명'].tolist()
name_list = name_list
name_list[:10]

['마포구 옛날집',
 '오발탄 서초점',
 '성동구 마장동한우촌',
 '마포구 태능식당',
 '성동구 육백더육백',
 '금천구 매꼬만 명태찜',
 '명동칼국수 본점',
 '인생설렁탕 장한평역점',
 '마포구 칭닌',
 '동대문구 청량리고기마을']

In [9]:
import requests

res_info_list = []
no_place = []

for searching in name_list:
    url = 'https://dapi.kakao.com/v2/local/search/keyword.json?query={}'.format(searching)
    headers = {
        "Authorization": "KakaoAK baccb3812bb7865995e93c318b31e10e"
    }
    places = requests.get(url, headers=headers).json()['documents']
    
    if len(places) <= 0:
        places = {'address_name': np.nan,
                 'category_group_code': np.nan,
                 'category_group_name': np.nan,
                 'category_name': np.nan,
                 'distance': np.nan,
                 'id': np.nan,
                 'phone': np.nan,
                 'place_name': np.nan,
                 'place_url': np.nan,
                 'road_address_name': np.nan,
                 'x': np.nan,
                 'y': np.nan}
        res_info_list.append(places)
        
    else:
        res_info_list.append(places[0])

In [ ]:
len(res_info_list)

In [ ]:
res_info_df = pd.DataFrame(res_info_list)
res_info_df['평점'] = raw['네이버평점']
res_info_df.loc[res_info_df['address_name'].isna(), '평점'] = np.nan

In [ ]:
res_info_df[:10]

In [ ]:
# 내 지역구가 아닌 지역구
not_my_area = ['강남구', '도봉구', '중구', '중랑구', '광진구', '종로구',
               '양천구', '용산구', '서대문구', '성북구', '송파구', '동작구', 
               '강서구', '노원구', '강동구', '영등포구', '강북구', '관악구', '은평구', '구로구']

# 내 지역구가 아닌 행의 인덱스를 찾음
rows_to_drop = res_info_df[res_info_df['address_name'].str.contains('|'.join(not_my_area), na=False)].index

# 해당 행을 삭제
res_info_df = res_info_df.drop(rows_to_drop)

# reset_index() 함수를 호출하여 인덱스를 다시 정렬
res_info_df = res_info_df.reset_index(drop=True)

In [ ]:
# 카카오API로 겁색하지 못한 음식점 개수
res_info_df.isna().sum()

In [ ]:
# 서울 아닌 행, NaN값 삭제
rows_to_drop2 = res_info_df[~res_info_df['address_name'].str.startswith('서울', na=False)].index
res_info_df = res_info_df.drop(rows_to_drop2)
res_info_df = res_info_df.reset_index(drop=True)

In [ ]:
res_info_df.info()


In [ ]:
#공통컬럼만
df_final = res_info_df.iloc[:,[0,3,7,8,10,11,12]]
df_final.columns = ['도로명주소', '카테고리', '상호명', '위도(y)', '경도(x)', '사이트', '별점']
df_final

In [ ]:
df_final['카테고리'].value_counts()

In [ ]:
#최종 데이터프레임
df_final.to_excel('식단품질정보_전처리완료.xlsx')